In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as ud
import numpy as np
import matplotlib.pyplot as plt
import torch.utils.data as tud
from PIL import Image
import torch.optim as optim
import torchvision.datasets as td

In [2]:
img_size = 256
imgpath = '/home/cv_lab/Jeni/imgs/'
img = Image.open(imgpath+'h1.jpeg')
#prep = transforms.Compose([transforms.Scale(img_size),
 #                          transforms.ToTensor()])
prep = transforms.Compose([transforms.CenterCrop(256), transforms.ToTensor()])
img_torch = prep(img)
img_var = Variable(img_torch.unsqueeze(0)).cuda()
#img_var = Variable(img_torch)
batch_size = 1

In [3]:
random_imgs_folder = '/home/daiict/CVShare/Jeni/random_imgs/'
random_imgs = td.ImageFolder(random_imgs_folder,transform = prep)
tot_imgs = (len(random_imgs))
loader = tud.DataLoader(random_imgs,shuffle=True)#,sampler=tud.sampler.RandomSampler)
print(len(loader))

32


In [4]:
'''for i,tmp in enumerate(loader):
    print(tmp[0].type)
    tmp = tmp[0].view(3,256,256)
    topil = transforms.ToPILImage()(tmp)
    plt.imshow(topil)
    plt.show()'''

'for i,tmp in enumerate(loader):\n    print(tmp[0].type)\n    tmp = tmp[0].view(3,256,256)\n    topil = transforms.ToPILImage()(tmp)\n    plt.imshow(topil)\n    plt.show()'

In [5]:
def sample_imgs(show='False'):
    num = np.random.randint(len(loader))
    sampler = loader.sampler
    nxtiter = sampler.__iter__()
    img1 = random_imgs.__getitem__(next(nxtiter))
    #print(img1[0].type)
    #print(img1.type())
    if(show=='True'):
        topil = transforms.ToPILImage()(img1[0])
        plt.imshow(topil)
        plt.show()
    return Variable(img1[0].unsqueeze(0)).cuda()

In [6]:
#_ = sample_imgs(show='True')

In [7]:
'''def sample_imgs(show='False'):
    sampler = tud.sampler.RandomSampler(random_imgs)
    nxtiter = sampler.__iter__()
    img,label = random_imgs.__getitem__(next(nxtiter))
    img = prep(img)
    if(show == 'True'):
        topil = transforms.ToPILImage()(img)
        plt.imshow(topil)
        plt.show()
    return Variable(img.unsqueeze(0))'''

"def sample_imgs(show='False'):\n    sampler = tud.sampler.RandomSampler(random_imgs)\n    nxtiter = sampler.__iter__()\n    img,label = random_imgs.__getitem__(next(nxtiter))\n    img = prep(img)\n    if(show == 'True'):\n        topil = transforms.ToPILImage()(img)\n        plt.imshow(topil)\n        plt.show()\n    return Variable(img.unsqueeze(0))"

In [8]:
class mycnn(nn.Module):
    def __init__(self):
        super(mycnn,self).__init__()
        self.cn1 = nn.Conv2d(img_channels,n_f1,3)
        self.max_pool = nn.MaxPool2d(3)
        self.cn2 = nn.Conv2d(n_f1,n_f2,3)
        
    def forward(self,x):
        x = F.relu(self.cn1(x))
        print(type(x))
        x = F.relu(self.cn2(x))
        x = self.max_pool(x)
        return x

In [9]:
class VGG(nn.Module):
    def __init__(self, pool='max'):
        super(VGG, self).__init__()
        #vgg modules
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_4 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_4 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_4 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.lin1 = nn.Linear(512*8*8,1024)
        self.lin2 = nn.Linear(1024,512)
        #self.lin3 = nn.Linear(512,512)
        self.lin4 = nn.Linear(512,128)
        self.lin5 = nn.Linear(128,1)
        if pool == 'max':
            self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)
        elif pool == 'avg':
            self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
            self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)
            self.pool3 = nn.AvgPool2d(kernel_size=2, stride=2)
            self.pool4 = nn.AvgPool2d(kernel_size=2, stride=2)
            self.pool5 = nn.AvgPool2d(kernel_size=2, stride=2)
            
    def forward(self, x):
        out = {}
        out['r11'] = F.relu(self.conv1_1(x))
        out['r12'] = F.relu(self.conv1_2(out['r11']))
        out['p1'] = self.pool1(out['r12'])
        out['r21'] = F.relu(self.conv2_1(out['p1']))
        out['r22'] = F.relu(self.conv2_2(out['r21']))
        out['p2'] = self.pool2(out['r22'])
        out['r31'] = F.relu(self.conv3_1(out['p2']))
        out['r32'] = F.relu(self.conv3_2(out['r31']))
        out['r33'] = F.relu(self.conv3_3(out['r32']))
        out['r34'] = F.relu(self.conv3_4(out['r33']))
        out['p3'] = self.pool3(out['r34'])
        out['r41'] = F.relu(self.conv4_1(out['p3']))
        out['r42'] = F.relu(self.conv4_2(out['r41']))
        out['r43'] = F.relu(self.conv4_3(out['r42']))
        out['r44'] = F.relu(self.conv4_4(out['r43']))
        out['p4'] = self.pool4(out['r44'])
        out['r51'] = F.relu(self.conv5_1(out['p4']))
        out['r52'] = F.relu(self.conv5_2(out['r51']))
        out['r53'] = F.relu(self.conv5_3(out['r52']))
        out['r54'] = F.relu(self.conv5_4(out['r53']))
        out['p5'] = self.pool5(out['r54'])
        
        #return [out[key] for key in out_keys]
        l = out['p5'].view(-1,512*8*8)
        l = F.relu(self.lin1(l))
        l = F.relu(self.lin2(l))
        #l = F.relu(self.lin3(l))
        l = F.relu(self.lin4(l))
        l = F.sigmoid(self.lin5(l))
        
        return l

In [10]:
vgg1 = VGG()
vgg1.cuda()
optimizer = optim.SGD(vgg1.parameters(), lr=0.001, momentum=0.9)

for ep in range(50):
    optimizer.zero_grad()
    true_op = vgg1(img_var)
    
    false_img = sample_imgs()
    false_op = vgg1(false_img)
    
    weights = torch.Tensor([5,0]).cuda()
    #criteria = nn.BCELoss(weight=weights)
    #cost = criteria(true_op)
    cost = -torch.log(true_op) - torch.log(1-false_op)
    cost.backward()
    optimizer.step()
    
    #if(ep%50==49):
    print('epoch', ep, 'cost', cost.data[0][0],'true_op', true_op.data[0][0],'false_op',false_op.data[0][0])

('epoch', 0, 'cost', 1.3878365755081177, 'true_op', 0.4803718626499176, 'false_op', 0.4803718626499176)
('epoch', 1, 'cost', 1.3878346681594849, 'true_op', 0.48038429021835327, 'false_op', 0.48038429021835327)
('epoch', 2, 'cost', 1.3878309726715088, 'true_op', 0.4804078936576843, 'false_op', 0.4804078936576843)
('epoch', 3, 'cost', 1.3878257274627686, 'true_op', 0.48044148087501526, 'false_op', 0.48044148087501526)
('epoch', 4, 'cost', 1.3878190517425537, 'true_op', 0.4804841876029968, 'false_op', 0.4804841876029968)
('epoch', 5, 'cost', 1.3878109455108643, 'true_op', 0.48053494095802307, 'false_op', 0.48053494095802307)
('epoch', 6, 'cost', 1.3878021240234375, 'true_op', 0.48059284687042236, 'false_op', 0.48059284687042236)
('epoch', 7, 'cost', 1.3877921104431152, 'true_op', 0.48065733909606934, 'false_op', 0.48065733909606934)
('epoch', 8, 'cost', 1.3877811431884766, 'true_op', 0.48072758316993713, 'false_op', 0.48072758316993713)
('epoch', 9, 'cost', 1.3877694606781006, 'true_op', 